This notebook will demonstrate feature engineering and augmentation for the GRU/LSTM model.

GRU/LSTM part is mainly based on [medalOpenVaccine: Simple GRU Model](https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model).


In [101]:
import pandas as pd
import numpy as np
import math, json, gc, random, os, sys
import json
import tensorflow.keras.layers as L
import keras.backend as K
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.cluster import KMeans
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
def allocate_gpu_memory(gpu_number=0):
    physical_devices = tf.config.experimental.list_physical_devices('GPU')

    if physical_devices:
        try:
            print("Found {} GPU(s)".format(len(physical_devices)))
            tf.config.set_visible_devices(physical_devices[gpu_number], 'GPU')
            tf.config.experimental.set_memory_growth(physical_devices[gpu_number], True)
            print("#{} GPU memory is allocated".format(gpu_number))
        except RuntimeError as e:
            print(e)
    else:
        print("Not enough GPU hardware devices available")
allocate_gpu_memory()

Ver='GRU_LSTM1'
aug_data = '../input/aug-data/aug_data1.csv'
debug = False

Found 1 GPU(s)
#0 GPU memory is allocated


In [102]:
SEED = 123
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(SEED)

In [103]:
def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return L.Bidirectional(L.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal'))

def build_model(seq_len=107, pred_len=68, dropout=0.5, embed_dim=100, hidden_dim=256, type=0):
    def cbr(x, out_layer, kernel, stride, dilation):
        x = L.Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        # x = L.BatchNormalization()(x)
        x = L.Activation("relu")(x)
        return x
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = L.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = L.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = L.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = L.Multiply()([tanh_out, sigm_out])
            x = L.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = L.Add()([res_x, x])
        return res_x
    
    inputs = L.Input(shape=(seq_len, 6))
    
    # split categorical and numerical features and concatenate them later.
    categorical_feat_dim = 3
    categorical_fea = inputs[:, :, :categorical_feat_dim]
    numerical_fea = inputs[:, :, 3:]

    embed = L.Embedding(input_dim=len(token2int), output_dim=embed_dim)(categorical_fea)
    reshaped = tf.reshape(embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3]))
    reshaped = L.concatenate([reshaped, numerical_fea], axis=2)
    
    if type == 0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    elif type == 1:
        hidden =cbr(reshaped, 64, 7, 1, 1)
        hidden = cbr(hidden, 64, 7, 1, 1)#
    elif type == 2:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
    elif type == 3:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    
    hidden = wave_block(hidden, 16, 3, 12)
    hidden = wave_block(hidden, 32, 3, 8)
    hidden = wave_block(hidden, 64, 3, 4)
    hidden = wave_block(hidden, 128, 3, 1)
    
    
    
    
    truncated = hidden[:, :pred_len]
    out = L.Dense(5, activation='linear')(truncated)
    model = tf.keras.Model(inputs=inputs, outputs=out)
    model.compile(tf.keras.optimizers.Adam(), loss=mcrmse)
    return model



In [104]:
token2int = {x:i for i, x in enumerate('().ACGUBEHIMSX')}
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

def preprocess_inputs(df, cols=['sequence', 'structure', 'predicted_loop_type']):
    base_fea = np.transpose(
        np.array(
            df[cols]
            .applymap(lambda seq: [token2int[x] for x in seq])
            .values
            .tolist()
        ),
        (0, 2, 1)
    )
    bpps_sum_fea = np.array(df['bpps_sum'].to_list())[:,:,np.newaxis]
    bpps_max_fea = np.array(df['bpps_max'].to_list())[:,:,np.newaxis]
    bpps_nb_fea = np.array(df['bpps_nb'].to_list())[:,:,np.newaxis]
    return np.concatenate([base_fea,bpps_sum_fea,bpps_max_fea,bpps_nb_fea], 2)

def rmse(y_actual, y_pred):
    mse = tf.keras.losses.mean_squared_error(y_actual, y_pred)
    return K.sqrt(mse)

def mcrmse(y_actual, y_pred, num_scored=len(pred_cols)):
    score = 0
    for i in range(num_scored):
        score += rmse(y_actual[:, :, i], y_pred[:, :, i]) / num_scored
    return score

## Load and preprocess data

In [105]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)

In [106]:
def read_bpps_sum(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").max(axis=1))
    return bpps_arr

def read_bpps_max(df):
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps_arr.append(np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy").sum(axis=1))
    return bpps_arr

def read_bpps_nb(df):
    # normalized non-zero number
    # from https://www.kaggle.com/symyksr/openvaccine-deepergcn 
    bpps_nb_mean = 0.077522 # mean of bpps_nb across all training data
    bpps_nb_std = 0.08914   # std of bpps_nb across all training data
    bpps_arr = []
    for mol_id in df.id.to_list():
        bpps = np.load(f"../input/stanford-covid-vaccine/bpps/{mol_id}.npy")
        bpps_nb = (bpps > 0).sum(axis=0) / bpps.shape[0]
        bpps_nb = (bpps_nb - bpps_nb_mean) / bpps_nb_std
        bpps_arr.append(bpps_nb)
    return bpps_arr 

train['bpps_sum'] = read_bpps_sum(train)
test['bpps_sum'] = read_bpps_sum(test)
train['bpps_max'] = read_bpps_max(train)
test['bpps_max'] = read_bpps_max(test)
train['bpps_nb'] = read_bpps_nb(train)
test['bpps_nb'] = read_bpps_nb(test)

In [107]:
# clustering for GroupKFold
# expecting more accurate CV by putting similar RNAs into the same fold.
kmeans_model = KMeans(n_clusters=200, random_state=110).fit(preprocess_inputs(train)[:,:,0])
train['cluster_id'] = kmeans_model.labels_

## Data augmentation for training and TTA(test)

In [108]:
aug_df = pd.read_csv(aug_data)
display(aug_df.head())

,id,sequence,structure,log_gamma,score,cnt,predicted_loop_type
0,id_fff546103,GGAAAGCUAGGACGUGGGAGCGUAGCUCUCCACACGGGUACGCCAA...,.....((((((((((((((((...)))).)))).((((((((((.....,2,0.981885,3,EEEEESSSSSSSSSSSSSSSSHHHSSSSBSSSSMSSSSSSSSSSHH...
1,id_18ff9d670,GGAAAGAGCUCGUGAGAAGAAUCUAGUACAUGCAUACGCUACAUCU...,.....(((.((.......((..((((.....((....)).....))...,0,0.887485,5,EEEEESSSISSIIIIIIISSIISSSSIIIIISSHHHHSSIIIIISS...
2,id_177cd630b,GGAAAGAAGUAGCACGGUCCUAAGGUUACUGUAGCUAUGUCCAGCG...,(....((.((((((((((((...))..))))).))))).))(((.(...,2,0.923722,3,SMMMMSSISSSSSSSSSSSSHHHSSBBSSSSSBSSSSSISSSSSIS...
3,id_17a9ad5b7,GGAAAACACUGCAAAAGUCAACGAAGAAGUUGACUAAGAAGUGAUC...,......((((((...(((((((......))))))).....((((((...,2,0.977602,3,EEEEEESSSSSSMMMSSSSSSSHHHHHHSSSSSSSMMMMMSSSSSS...
4,id_17ab91518,GGAAAACGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCG...,......(((((((((((((((((((((((((((((....)))))))...,2,0.982851,3,EEEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSS...


This file was created using ARNIE, ViennaRNA and bpRNA in the following way.

Get candidate structures with different gamma values. 
See last cell of [How to Use ARNIE on Kaggle Notebook](https://www.kaggle.com/its7171/how-to-use-arnie-on-kaggle-notebook).

Get candidate structures with different gamma values.

Remove the same as the original structure.


Get a structure with the largest score for each sequence.

Get the predicted_loop_type from the sequence and structure.
See [How to Generate predicted_loop_type](https://www.kaggle.com/its7171/how-to-generate-predicted-loop-type).

In [109]:
LR = 0.001

In [110]:
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import BatchNormalization

class SWA(Callback):
    """ Stochastic Weight Averging.
    # Paper
        title: Averaging Weights Leads to Wider Optima and Better Generalization
        link: https://arxiv.org/abs/1803.05407
    # Arguments
        start_epoch:   integer, epoch when swa should start.
        lr_schedule:   string, type of learning rate schedule.
        swa_lr:        float, learning rate for swa sampling.
        swa_lr2:       float, upper bound of cyclic learning rate.
        swa_freq:      integer, length of learning rate cycle.
        verbose:       integer, verbosity mode, 0 or 1.
    """
    def __init__(self,
                 start_epoch,
                 no_samples,
                 lr_schedule='manual',
                 swa_lr='auto',
                 swa_lr2='auto',
                 swa_freq=1,
                 verbose=0):
                 
        super(SWA, self).__init__()
        self.start_epoch = start_epoch - 1
        self.lr_schedule = lr_schedule
        self.swa_lr = swa_lr
        self.swa_lr2 = swa_lr2
        self.swa_freq = swa_freq
        self.verbose = verbose
        self.no_samples = no_samples
        # self.params['samples'] = no_samples 

        if start_epoch < 2:
            raise ValueError('"swa_start" attribute cannot be lower than 2.')

        schedules = ['manual', 'constant', 'cyclic']

        if self.lr_schedule not in schedules:
            raise ValueError('"{}" is not a valid learning rate schedule' \
                             .format(self.lr_schedule))

        if self.lr_schedule == 'cyclic' and self.swa_freq < 2:
            raise ValueError('"swa_freq" must be higher than 1 for cyclic schedule.')

        if self.swa_lr == 'auto' and self.swa_lr2 != 'auto':
            raise ValueError('"swa_lr2" cannot be manually set if "swa_lr" is automatic.') 
            
        if self.lr_schedule == 'cyclic' and self.swa_lr != 'auto' \
           and self.swa_lr2 != 'auto' and self.swa_lr > self.swa_lr2:
            raise ValueError('"swa_lr" must be lower than "swa_lr2".')

    def on_train_begin(self, logs=None):

        self.epochs = self.params.get('epochs')

        if self.start_epoch >= self.epochs - 1:
            raise ValueError('"swa_start" attribute must be lower than "epochs".')

        self.init_lr = K.eval(self.model.optimizer.lr)

        # automatic swa_lr
        if self.swa_lr == 'auto':
            self.swa_lr = 0.1*self.init_lr
        
        if self.init_lr < self.swa_lr:
            raise ValueError('"swa_lr" must be lower than rate set in optimizer.')

        # automatic swa_lr2 between initial lr and swa_lr   
        if self.lr_schedule == 'cyclic' and self.swa_lr2 == 'auto':
            self.swa_lr2 = self.swa_lr + (self.init_lr - self.swa_lr)*0.25

        self._check_batch_norm()

    def on_epoch_begin(self, epoch, logs=None):

        self.current_epoch = epoch
        self._scheduler(epoch)

        # constant schedule is updated epoch-wise
        if self.lr_schedule == 'constant' or self.is_batch_norm_epoch:
            self._update_lr(epoch)

        if self.is_swa_start_epoch:
            self.swa_weights = self.model.get_weights()

            if self.verbose > 0:
                print('\nEpoch %05d: starting stochastic weight averaging'
                      % (epoch + 1))

        if self.is_batch_norm_epoch:
            self._set_swa_weights(epoch)

            if self.verbose > 0:
                print('\nEpoch %05d: reinitializing batch normalization layers'
                      % (epoch + 1))

            self._reset_batch_norm()

            if self.verbose > 0:
                print('\nEpoch %05d: running forward pass to adjust batch normalization'
                      % (epoch + 1))

    def on_batch_begin(self, batch, logs=None):

        # update lr each batch for cyclic lr schedule
        if self.lr_schedule == 'cyclic':
            self._update_lr(self.current_epoch, batch)

        if self.is_batch_norm_epoch:
            batch_size = 64
            momentum = batch_size / (batch*batch_size + batch_size)

            for layer in self.batch_norm_layers:
                layer.momentum = momentum

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        logs['lr'] = K.eval(self.model.optimizer.lr)
        for k, v in logs.items():
            if k == 'lr':
                self.model.history.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):

        if self.is_swa_start_epoch:
            self.swa_start_epoch = epoch

        if self.is_swa_epoch and not self.is_batch_norm_epoch:
            self.swa_weights = self._average_weights(epoch)

    def on_train_end(self, logs=None):

        if not self.has_batch_norm:
            self._set_swa_weights(self.epochs)
        else:
            self._restore_batch_norm()

    def _scheduler(self, epoch):

        swa_epoch = (epoch - self.start_epoch)

        self.is_swa_epoch = epoch >= self.start_epoch and swa_epoch % self.swa_freq == 0
        self.is_swa_start_epoch = epoch == self.start_epoch
        self.is_batch_norm_epoch = epoch == self.epochs - 1 and self.has_batch_norm

    def _average_weights(self, epoch):

        return [(swa_w * (epoch - self.start_epoch) + w)
                / ((epoch - self.start_epoch) + 1)
                for swa_w, w in zip(self.swa_weights, self.model.get_weights())]

    def _update_lr(self, epoch, batch=None):

        if self.is_batch_norm_epoch:
            lr = 0
            K.set_value(self.model.optimizer.lr, lr)
        elif self.lr_schedule == 'constant':
            lr = self._constant_schedule(epoch)
            K.set_value(self.model.optimizer.lr, lr)
        elif self.lr_schedule == 'cyclic':
            lr = self._cyclic_schedule(epoch, batch)
            K.set_value(self.model.optimizer.lr, lr)

    def _constant_schedule(self, epoch):

        t = epoch / self.start_epoch
        lr_ratio = self.swa_lr / self.init_lr
        if t <= 0.5:
            factor = 1.0
        elif t <= 0.9:
            factor = 1.0 - (1.0 - lr_ratio) * (t - 0.5) / 0.4
        else:
            factor = lr_ratio
        return self.init_lr * factor

    def _cyclic_schedule(self, epoch, batch):
        """ Designed after Section 3.1 of Averaging Weights Leads to
        Wider Optima and Better Generalization(https://arxiv.org/abs/1803.05407)
        """
        # steps are mini-batches per epoch, equal to training_samples / batch_size
        steps = self.params.get('steps')
        
        #occasionally steps parameter will not be set. We then calculate it ourselves
        if steps == None:
            self.params['samples'] = self.no_samples
            steps = self.params['samples'] // self.params['batch_size']
        
        swa_epoch = (epoch - self.start_epoch) % self.swa_freq
        cycle_length = self.swa_freq * steps

        # batch 0 indexed, so need to add 1
        i = (swa_epoch * steps) + (batch + 1)
        if epoch >= self.start_epoch:
            t = (((i-1) % cycle_length) + 1)/cycle_length
            return (1-t)*self.swa_lr2 + t*self.swa_lr
        else:
            return self._constant_schedule(epoch)

    def _set_swa_weights(self, epoch):

        self.model.set_weights(self.swa_weights)

        if self.verbose > 0:
            print('\nEpoch %05d: final model weights set to stochastic weight average'
                  % (epoch + 1))

    def _check_batch_norm(self):

        self.batch_norm_momentums = []
        self.batch_norm_layers = []
        self.has_batch_norm = False
        self.running_bn_epoch = False

        for layer in self.model.layers:
            if issubclass(layer.__class__, BatchNormalization):
                self.has_batch_norm = True
                self.batch_norm_momentums.append(layer.momentum)
                self.batch_norm_layers.append(layer)

        if self.verbose > 0 and self.has_batch_norm:
            print('Model uses batch normalization. SWA will require last epoch '
                  'to be a forward pass and will run with no learning rate')

    def _reset_batch_norm(self):

        for layer in self.batch_norm_layers:

            # to get properly initialized moving mean and moving variance weights
            # we initialize a new batch norm layer from the config of the existing
            # layer, build that layer, retrieve its reinitialized moving mean and
            # moving var weights and then delete the layer
            bn_config = layer.get_config()
            new_batch_norm = BatchNormalization(**bn_config)
            new_batch_norm.build(layer.input_shape)
            new_moving_mean, new_moving_var = new_batch_norm.get_weights()[-2:]
            # get rid of the new_batch_norm layer
            del new_batch_norm
            # get the trained gamma and beta from the current batch norm layer
            trained_weights = layer.get_weights()
            new_weights = []
            # get gamma if exists
            if bn_config['scale']:
                new_weights.append(trained_weights.pop(0))
            # get beta if exists
            if bn_config['center']:
                new_weights.append(trained_weights.pop(0))
            new_weights += [new_moving_mean, new_moving_var]
            # set weights to trained gamma and beta, reinitialized mean and variance
            layer.set_weights(new_weights)

    def _restore_batch_norm(self):

        for layer, momentum in zip(self.batch_norm_layers, self.batch_norm_momentums):
            layer.momentum = momentum

In [111]:
def lr_schedule(epoch):
    if epoch < 30:
        lr = LR
    elif epoch < 40:
        lr = LR / 3
    elif epoch < 50:
        lr = LR / 5
    elif epoch < 60:
        lr = LR / 7
    elif epoch < 70:
        lr = LR / 9
    elif epoch < 80:
        lr = LR / 11
    elif epoch < 90:
        lr = LR / 13
    elif epoch < 120:
        lr = LR / 15
    else:
        lr = LR / 17
    return lr

In [112]:
def aug_data(df):
    target_df = df.copy()
    new_df = aug_df[aug_df['id'].isin(target_df['id'])]
                         
    del target_df['structure']
    del target_df['predicted_loop_type']
    new_df = new_df.merge(target_df, on=['id','sequence'], how='left')

    df['cnt'] = df['id'].map(new_df[['id','cnt']].set_index('id').to_dict()['cnt'])
    df['log_gamma'] = 100
    df['score'] = 1.0
    df = df.append(new_df[df.columns])
    return df
train = aug_data(train)
test = aug_data(test)

In [113]:
if debug:
    train = train[:200]
    test = test[:200]

## Build and train model

In [114]:
model = build_model()
model.summary()

Model: "functional_87"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(None, 107, 6)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_129 ( [(None, 107, 3)]     0           input_44[0][0]                   
__________________________________________________________________________________________________
embedding_43 (Embedding)        (None, 107, 3, 100)  1400        tf_op_layer_strided_slice_129[0][
__________________________________________________________________________________________________
tf_op_layer_Reshape_43 (TensorF [(None, 107, 300)]   0           embedding_43[0][0]               
______________________________________________________________________________________

In [115]:
def train_and_predict(type = 0, FOLD_N = 2):
    
    gkf = GroupKFold(n_splits=FOLD_N)

    public_df = test.query("seq_length == 107").copy()
    private_df = test.query("seq_length == 130").copy()

    public_inputs = preprocess_inputs(public_df)
    private_inputs = preprocess_inputs(private_df)


    holdouts = []
    holdout_preds = []

    for cv, (tr_idx, vl_idx) in enumerate(gkf.split(train,  train['reactivity'], train['cluster_id'])):
        trn = train.iloc[tr_idx]
        x_trn = preprocess_inputs(trn)
        y_trn = np.array(trn[pred_cols].values.tolist()).transpose((0, 2, 1))
        w_trn = np.log(trn.signal_to_noise+1.1)/2

        val = train.iloc[vl_idx]
        x_val_all = preprocess_inputs(val)
        val = val[val.SN_filter == 1]
        x_val = preprocess_inputs(val)
        y_val = np.array(val[pred_cols].values.tolist()).transpose((0, 2, 1))

        model = build_model(type=type)
        model_short = build_model(seq_len=107, pred_len=107,type=type)
        model_long = build_model(seq_len=130, pred_len=130,type=type)
        
        
        
        start_epoch = 30

        #define swa callback
        swa = SWA(start_epoch=start_epoch,
                  no_samples=x_trn.shape[0],
                  lr_schedule='manual',
                  swa_lr=0.001,
                  verbose=1)

        history = model.fit(
            x_trn, y_trn,
            validation_data = (x_val, y_val),
            batch_size=64,
            epochs=60,   # 80,
            sample_weight=w_trn,
            callbacks=[
                tf.keras.callbacks.ReduceLROnPlateau(),
                swa,
                tf.keras.callbacks.ModelCheckpoint(f'model{Ver}_cv{cv}.h5',save_best_only=True)
            ]
        )

#         fig = px.line(
#             history.history, y=['loss', 'val_loss'], 
#             labels={'index': 'epoch', 'value': 'Mean Squared Error'}, 
#             title='Training History')
#         fig.show()

        model.load_weights(f'model{Ver}_cv{cv}.h5')
        model_short.load_weights(f'model{Ver}_cv{cv}.h5')
        model_long.load_weights(f'model{Ver}_cv{cv}.h5')

        holdouts.append(train.iloc[vl_idx])
        holdout_preds.append(model.predict(x_val_all))
        if cv == 0:
            public_preds = model_short.predict(public_inputs)/FOLD_N
            private_preds = model_long.predict(private_inputs)/FOLD_N
        else:
            public_preds += model_short.predict(public_inputs)/FOLD_N
            private_preds += model_long.predict(private_inputs)/FOLD_N
    return holdouts, holdout_preds, public_df, public_preds, private_df, private_preds

In [116]:
val_df, val_preds, test_df, test_preds = [], [], [], []
if debug:
    nmodel = 1
else:
    nmodel = 4
for i in range(nmodel):
    holdouts, holdout_preds, public_df, public_preds, private_df, private_preds = train_and_predict(i)
    val_df += holdouts
    val_preds += holdout_preds
    test_df.append(public_df)
    test_df.append(private_df)
    test_preds.append(public_preds)
    test_preds.append(private_preds)

Epoch 1/60
38/38 [==============================] - 10s 262ms/step - loss: 0.3363 - val_loss: 0.3371
Epoch 2/60
38/38 [==============================] - 8s 204ms/step - loss: 0.2771 - val_loss: 0.3079
Epoch 3/60
38/38 [==============================] - 8s 204ms/step - loss: 0.2604 - val_loss: 0.2960
Epoch 4/60
38/38 [==============================] - 8s 205ms/step - loss: 0.2501 - val_loss: 0.2864
Epoch 5/60
38/38 [==============================] - 8s 203ms/step - loss: 0.2411 - val_loss: 0.2772
Epoch 6/60
38/38 [==============================] - 8s 211ms/step - loss: 0.2310 - val_loss: 0.2659
Epoch 7/60
38/38 [==============================] - 8s 205ms/step - loss: 0.2216 - val_loss: 0.2643
Epoch 8/60
38/38 [==============================] - 8s 208ms/step - loss: 0.2123 - val_loss: 0.2562
Epoch 9/60
38/38 [==============================] - 8s 203ms/step - loss: 0.2074 - val_loss: 0.2554
Epoch 10/60
38/38 [==============================] - 8s 204ms/step - loss: 0.2003 - val_loss: 0.247

KeyboardInterrupt: 

## post process

In [ ]:
preds_ls = []
for df, preds in zip(test_df, test_preds):
    for i, uid in enumerate(df.id):
        single_pred = preds[i]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
preds_df = pd.concat(preds_ls).groupby('id_seqpos').mean().reset_index()
# .mean() is for
# 1, Predictions from multiple models
# 2, TTA (augmented test data)

preds_ls = []
for df, preds in zip(val_df, val_preds):
    for i, uid in enumerate(df.id):
        single_pred = preds[i]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        single_df['SN_filter'] = df[df['id'] == uid].SN_filter.values[0]
        preds_ls.append(single_df)
holdouts_df = pd.concat(preds_ls).groupby('id_seqpos').mean().reset_index()

In [ ]:
submission = preds_df[['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']]
submission.to_csv(f'submission.csv', index=False)
print(f'wrote to submission.csv')

## Validation

In [ ]:
def print_mse(prd):
    val = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)

    val_data = []
    for mol_id in val['id'].unique():
        sample_data = val.loc[val['id'] == mol_id]
        sample_seq_length = sample_data.seq_length.values[0]
        for i in range(68):
            sample_dict = {
                           'id_seqpos' : sample_data['id'].values[0] + '_' + str(i),
                           'reactivity_gt' : sample_data['reactivity'].values[0][i],
                           'deg_Mg_pH10_gt' : sample_data['deg_Mg_pH10'].values[0][i],
                           'deg_Mg_50C_gt' : sample_data['deg_Mg_50C'].values[0][i],
                           }
            val_data.append(sample_dict)
    val_data = pd.DataFrame(val_data)
    val_data = val_data.merge(prd, on='id_seqpos')

    rmses = []
    mses = []
    for col in ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']:
        rmse = ((val_data[col] - val_data[col+'_gt']) ** 2).mean() ** .5
        mse = ((val_data[col] - val_data[col+'_gt']) ** 2).mean()
        rmses.append(rmse)
        mses.append(mse)
        print(col, rmse, mse)
    print(np.mean(rmses), np.mean(mses))

In [ ]:
print_mse(holdouts_df)

In [ ]:
print_mse(holdouts_df[holdouts_df.SN_filter == 1])